In [1]:
# ===============================================
# 📦 Cell 1: Setup Environment
# ===============================================
import os, warnings, time
warnings.filterwarnings('ignore')

# Create project directories
os.makedirs('medical_rag_bot/docs', exist_ok=True)
os.chdir('/content/medical_rag_bot')

# Install required packages
!pip install -q google-generativeai langchain langchain-google-genai chromadb PyMuPDF langchain-community

print("✅ Environment ready! Docs folder: medical_rag_bot/docs")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.9 MB/s eta 0:00:

In [2]:
# ===============================================
# 🔑 Cell 2: API Configuration & Imports
# ===============================================
from google.colab import userdata
import google.generativeai as genai
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configure Gemini API
try:
    API_KEY = userdata.get('MY_GEMINI_KEY')
    os.environ["GOOGLE_API_KEY"] = API_KEY
    genai.configure(api_key=API_KEY)
    print("✅ Gemini API configured!")
except Exception as e:
    print("❌ API Key Error:", e)


✅ Gemini API configured!


In [3]:
# ===============================================
# 📥 Cell 3: Document Upload
# ===============================================
from google.colab import files
import shutil

print("📋 Upload your medical PDFs")

uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        shutil.move(filename, f'docs/{filename}')
        print(f"✅ Uploaded: {filename}")


📋 Upload your medical PDFs


Saving Martha Grace Kommuguri_NR2.pdf to Martha Grace Kommuguri_NR2.pdf
✅ Uploaded: Martha Grace Kommuguri_NR2.pdf


In [4]:
# ===============================================
# 📖 Cell 4: Document Processing
# ===============================================
print("📖 Processing documents...")
documents = []

for filename in os.listdir("docs"):
    if filename.endswith(".pdf"):
        loader = PyMuPDFLoader(f"docs/{filename}")
        docs = loader.load()
        documents.extend(docs)
        print(f"📄 {filename}: {len(docs)} pages")

if documents:
    splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
    chunks = splitter.split_documents(documents)
    print(f"✅ Split into {len(chunks)} chunks")
else:
    print("❌ No documents found!")


📖 Processing documents...
📄 Martha Grace Kommuguri_NR2.pdf: 1 pages
✅ Split into 4 chunks


In [5]:
# ===============================================
# 🧬 Cell 5: Vector Database Setup
# ===============================================
print("🧬 Creating Chroma Vector DB...")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectordb = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory="medical_db"
)

retriever = vectordb.as_retriever(search_kwargs={"k": 5})

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)

print("✅ Vector DB + LLM ready!")


🧬 Creating Chroma Vector DB...
✅ Vector DB + LLM ready!


In [6]:
# ===============================================
# 🤖 Cell 6: Hybrid Q&A Functions
# ===============================================

def ask_llm_first(query):
    """Try LLM first. If unsure → fallback."""
    system_prompt = f"""
    You are a medical AI assistant. Answer only if you are confident.
    If you don’t know, reply exactly: "I don't know."

    Question: {query}
    """
    response = llm.invoke(system_prompt)
    content = response.content if hasattr(response, "content") else str(response)

    if "i don't know" in content.lower():
        return None
    return content

def ask_with_rag(query):
    """Use RAG retrieval if LLM unsure."""
    retrieved_docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs[:3]])

    rag_prompt = f"""
    Use the encyclopedia content below to answer:

    {context}

    Question: {query}
    """
    response = llm.invoke(rag_prompt)
    return response.content if hasattr(response, "content") else str(response)

def hybrid_query(query):
    """Main function: LLM → fallback to RAG."""
    llm_ans = ask_llm_first(query)
    if llm_ans:
        return f"🧠 DIRECT: {llm_ans}"
    else:
        rag_ans = ask_with_rag(query)
        return f"📚 RAG: {rag_ans}"


In [7]:
# ===============================================
# 💬cell 7 Interactive Chat
# ===============================================

def start_chat():
    print("🏥 Medical Chatbot (Type 'exit' to quit)\n")
    while True:
        user_q = input("💬 You: ").strip()
        if user_q.lower() == "exit":
            print("👋 Goodbye!")
            break
        answer = hybrid_query(user_q)
        print(f"\n🤖 {answer}\n")


In [8]:
start_chat()

🏥 Medical Chatbot (Type 'exit' to quit)

💬 You: what is the pdf about


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 47
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 45
}
]